In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
import random

In [2]:
batch_size=256
epochs=300
seed=1
cuda=False and torch.cuda.is_available()
log_interval=1
r_dim=128
z_dim=128


In [3]:

torch.manual_seed(seed)
random.seed(seed)
device = torch.device("cpu") #"cuda" if args.cuda else 

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)


In [4]:
def get_context_idx(N):
    # generate the indeces of the N context points in a flattened image
    idx = random.sample(range(0, 784), N)
    idx = torch.tensor(idx, device=device)
    return idx

In [5]:
def generate_grid(h, w):
    rows = torch.linspace(0, 1, h, device=device)
    cols = torch.linspace(0, 1, w, device=device)
    grid = torch.stack([cols.repeat(h, 1).t().contiguous().view(-1), rows.repeat(w)], dim=1)
    grid = grid.unsqueeze(0)
    return grid

In [6]:
def idx_to_y(idx, data):
    # get the [0;1] pixel intensity at each index
    y = torch.index_select(data, dim=1, index=idx)
    return y


In [7]:
def idx_to_x(idx, batch_size):
    # From flat idx to 2d coordinates of the 28x28 grid. E.g. 35 -> (1, 7)
    # Equivalent to np.unravel_index()
    x = torch.index_select(x_grid, dim=1, index=idx)
    x = x.expand(batch_size, -1, -1)
    return x

In [16]:
class CNP(nn.Module):
    def __init__(self, r_dim,z_dim):
        super(CNP, self).__init__()
        self.r_dim = r_dim
        self.z_dim = z_dim
        self.h_1 = nn.Linear(3, 256)
        self.h_2 = nn.Linear(256, 256)
        self.h_3 = nn.Linear(256, self.r_dim)

        self.r_to_z_mean = nn.Linear(self.r_dim, self.z_dim)
        self.r_to_z_logvar = nn.Linear(self.r_dim, self.z_dim)

        self.g_1 = nn.Linear(self.z_dim + 2, 256)
        self.g_2 = nn.Linear(256,256)
        self.g_3 = nn.Linear(256,256)
        self.g_mu = nn.Linear(256, 1)
        self.g_sigma = nn.Linear(256, 1)

    def h(self, x_y):
        x_y = F.relu(self.h_1(x_y))
        x_y = F.relu(self.h_2(x_y))
        x_y = F.relu(self.h_3(x_y))
        return x_y

    def aggregate(self, r):
        return torch.mean(r, dim=1)

    def g(self, rep, x_target):
        r_et_x= torch.cat([rep, x_target], dim=2)
        input = F.relu(self.g_1(r_et_x))
        input = F.relu(self.g_2(input))
        input = F.relu(self.g_3(input))
        mu=self.g_mu(input)
        log_sigma=self.g_sigma(input)
        return mu,log_sigma

    def xy_to_r_params(self, x, y):
        
        x_y = torch.cat([x, y], dim=2)
        
        r_i = self.h(x_y)
        r = self.aggregate(r_i)
        
        return r

    def forward(self, x_context, y_context,inTest=False):
       
        r_context = self.xy_to_r_params(x_context, y_context)  # (mu, logvar) of z
        
        r_expand = r_context.unsqueeze(1).expand(-1, 784, -1)

        # reconstruct the whole image including the provided context points
        
        x_target = x_grid.expand(y_context.shape[0], -1, -1)
        
        #change r with a random tensor in test phase
        if inTest:
            r_expand=torch.rand(r_expand.size())
        
        mu, log_sigma = self.g(r_expand, x_target)
    
        # Bound the variance
        sigma = 0.1 + 0.9 * F.softplus(log_sigma)

        return mu, sigma
        

In [9]:
def np_loss(mu,sigma, y):#, z_all, z_context

    if y is not None:
        log_p = -(y - mu)**2 / (2 * sigma**2) - torch.log(sigma)
    else:
        log_p = None
    return log_p

In [10]:
model = CNP(r_dim,z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
x_grid = generate_grid(28, 28)
os.makedirs("results_cnp_r_alea/", exist_ok=True)

In [11]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (y_all, _) in enumerate(train_loader):
        
        batch_size = y_all.shape[0]
        
        y_all = y_all.to(device).view(batch_size, -1, 1)
        
        N = random.randint(1, 784)  # number of context points
        context_idx = get_context_idx(N)
        x_context = idx_to_x(context_idx, batch_size)
        
        y_context = idx_to_y(context_idx, y_all)
        optimizer.zero_grad()
        
        mu,sigma= model(x_context, y_context)
        
        loss = -np_loss(mu,sigma, y_all).sum(dim=0).mean() #z_all, z_context
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(y_all), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(y_all)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

In [12]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (y_all, _) in enumerate(test_loader):
            y_all = y_all.to(device).view(y_all.shape[0], -1, 1)
            batch_size = y_all.shape[0]

            N = 300
            context_idx = get_context_idx(N)
            x_context = idx_to_x(context_idx, batch_size)
            y_context = idx_to_y(context_idx, y_all)

            mu,sigma = model(x_context, y_context,inTest=True)
            test_loss += -np_loss(mu,sigma, y_all).sum(dim=0).mean().item()

            if i == 0:  # save PNG of reconstructed examples
                plot_Ns = [10, 100, 300, 784]
                num_examples = min(batch_size, 16)
                for N in plot_Ns:
                    recons = []
                    recons1=[]
                    context_idx = get_context_idx(N)
                    x_context = idx_to_x(context_idx, batch_size)
                    y_context = idx_to_y(context_idx, y_all)
                    for d in range(5):#r is a random tensor, output 5 times to see its influence
                        mu,sigma= model(x_context, y_context)
                        recons.append(mu[:num_examples])
                        recons1.append(sigma[:num_examples])
                    recons = torch.cat(recons).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    recons1 = torch.cat(recons1).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    background = torch.tensor([0., 0., 1.], device=device)
                    background = background.view(1, -1, 1).expand(num_examples, 3, 784).contiguous()
                    context_pixels = y_all[:num_examples].view(num_examples, 1, -1)[:, :, context_idx]
                    context_pixels = context_pixels.expand(num_examples, 3, -1)
                    background[:, :, context_idx] = context_pixels
                    comparison = torch.cat([background.view(-1, 3, 28, 28),
                                            recons,recons1])
                    save_image(comparison.cpu(),
                               'results_cnp_r_alea/ep_' + str(epoch) +
                               '_cps_' + str(N) + '.png', nrow=num_examples)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [13]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: -0.256158
Train Epoch: 1 [256/60000 (0%)]	Loss: -0.260415
Train Epoch: 1 [512/60000 (1%)]	Loss: -0.260819
Train Epoch: 1 [768/60000 (1%)]	Loss: -0.262615
Train Epoch: 1 [1024/60000 (2%)]	Loss: -0.265586
Train Epoch: 1 [1280/60000 (2%)]	Loss: -0.271199
Train Epoch: 1 [1536/60000 (3%)]	Loss: -0.276390
Train Epoch: 1 [1792/60000 (3%)]	Loss: -0.282217
Train Epoch: 1 [2048/60000 (3%)]	Loss: -0.277220
Train Epoch: 1 [2304/60000 (4%)]	Loss: -0.283102
Train Epoch: 1 [2560/60000 (4%)]	Loss: -0.284454
Train Epoch: 1 [2816/60000 (5%)]	Loss: -0.290498
Train Epoch: 1 [3072/60000 (5%)]	Loss: -0.295663
Train Epoch: 1 [3328/60000 (6%)]	Loss: -0.303871
Train Epoch: 1 [3584/60000 (6%)]	Loss: -0.301341
Train Epoch: 1 [3840/60000 (6%)]	Loss: -0.309204
Train Epoch: 1 [4096/60000 (7%)]	Loss: -0.307859
Train Epoch: 1 [4352/60000 (7%)]	Loss: -0.317019
Train Epoch: 1 [4608/60000 (8%)]	Loss: -0.319146
Train Epoch: 1 [4864/60000 (8%)]	Loss: -0.321005
Train Epoch: 1 [5120/60000

Train Epoch: 1 [41472/60000 (69%)]	Loss: -0.679150
Train Epoch: 1 [41728/60000 (69%)]	Loss: -0.683741
Train Epoch: 1 [41984/60000 (70%)]	Loss: -0.690417
Train Epoch: 1 [42240/60000 (70%)]	Loss: -0.699462
Train Epoch: 1 [42496/60000 (71%)]	Loss: -0.674031
Train Epoch: 1 [42752/60000 (71%)]	Loss: -0.684594
Train Epoch: 1 [43008/60000 (71%)]	Loss: -0.680494
Train Epoch: 1 [43264/60000 (72%)]	Loss: -0.679808
Train Epoch: 1 [43520/60000 (72%)]	Loss: -0.692021
Train Epoch: 1 [43776/60000 (73%)]	Loss: -0.677428
Train Epoch: 1 [44032/60000 (73%)]	Loss: -0.665446
Train Epoch: 1 [44288/60000 (74%)]	Loss: -0.679874
Train Epoch: 1 [44544/60000 (74%)]	Loss: -0.674791
Train Epoch: 1 [44800/60000 (74%)]	Loss: -0.674426
Train Epoch: 1 [45056/60000 (75%)]	Loss: -0.680201
Train Epoch: 1 [45312/60000 (75%)]	Loss: -0.695521
Train Epoch: 1 [45568/60000 (76%)]	Loss: -0.699983
Train Epoch: 1 [45824/60000 (76%)]	Loss: -0.673511
Train Epoch: 1 [46080/60000 (77%)]	Loss: -0.690299
Train Epoch: 1 [46336/60000 (77

Train Epoch: 2 [22528/60000 (37%)]	Loss: -0.729524
Train Epoch: 2 [22784/60000 (38%)]	Loss: -0.727664
Train Epoch: 2 [23040/60000 (38%)]	Loss: -0.724559
Train Epoch: 2 [23296/60000 (39%)]	Loss: -0.741568
Train Epoch: 2 [23552/60000 (39%)]	Loss: -0.730225
Train Epoch: 2 [23808/60000 (40%)]	Loss: -0.757089
Train Epoch: 2 [24064/60000 (40%)]	Loss: -0.718771
Train Epoch: 2 [24320/60000 (40%)]	Loss: -0.727346
Train Epoch: 2 [24576/60000 (41%)]	Loss: -0.749084
Train Epoch: 2 [24832/60000 (41%)]	Loss: -0.736826
Train Epoch: 2 [25088/60000 (42%)]	Loss: -0.737318
Train Epoch: 2 [25344/60000 (42%)]	Loss: -0.727023
Train Epoch: 2 [25600/60000 (43%)]	Loss: -0.730753
Train Epoch: 2 [25856/60000 (43%)]	Loss: -0.729685
Train Epoch: 2 [26112/60000 (43%)]	Loss: -0.735434
Train Epoch: 2 [26368/60000 (44%)]	Loss: -0.726507
Train Epoch: 2 [26624/60000 (44%)]	Loss: -0.752896
Train Epoch: 2 [26880/60000 (45%)]	Loss: -0.747910
Train Epoch: 2 [27136/60000 (45%)]	Loss: -0.754859
Train Epoch: 2 [27392/60000 (46

Train Epoch: 3 [3328/60000 (6%)]	Loss: -0.826238
Train Epoch: 3 [3584/60000 (6%)]	Loss: -0.849619
Train Epoch: 3 [3840/60000 (6%)]	Loss: -0.854484
Train Epoch: 3 [4096/60000 (7%)]	Loss: -0.831487
Train Epoch: 3 [4352/60000 (7%)]	Loss: -0.871556
Train Epoch: 3 [4608/60000 (8%)]	Loss: -0.850728
Train Epoch: 3 [4864/60000 (8%)]	Loss: -0.860338
Train Epoch: 3 [5120/60000 (9%)]	Loss: -0.845551
Train Epoch: 3 [5376/60000 (9%)]	Loss: -0.852657
Train Epoch: 3 [5632/60000 (9%)]	Loss: -0.857720
Train Epoch: 3 [5888/60000 (10%)]	Loss: -0.858646
Train Epoch: 3 [6144/60000 (10%)]	Loss: -0.862722
Train Epoch: 3 [6400/60000 (11%)]	Loss: -0.855121
Train Epoch: 3 [6656/60000 (11%)]	Loss: -0.844412
Train Epoch: 3 [6912/60000 (11%)]	Loss: -0.861014
Train Epoch: 3 [7168/60000 (12%)]	Loss: -0.864375
Train Epoch: 3 [7424/60000 (12%)]	Loss: -0.879884
Train Epoch: 3 [7680/60000 (13%)]	Loss: -0.854623
Train Epoch: 3 [7936/60000 (13%)]	Loss: -0.870612
Train Epoch: 3 [8192/60000 (14%)]	Loss: -0.851700
Train Epoc

Train Epoch: 3 [44800/60000 (74%)]	Loss: -0.959794
Train Epoch: 3 [45056/60000 (75%)]	Loss: -0.973839
Train Epoch: 3 [45312/60000 (75%)]	Loss: -0.965503
Train Epoch: 3 [45568/60000 (76%)]	Loss: -0.975808
Train Epoch: 3 [45824/60000 (76%)]	Loss: -0.982350
Train Epoch: 3 [46080/60000 (77%)]	Loss: -0.961185
Train Epoch: 3 [46336/60000 (77%)]	Loss: -0.976079
Train Epoch: 3 [46592/60000 (77%)]	Loss: -0.979874
Train Epoch: 3 [46848/60000 (78%)]	Loss: -0.955345
Train Epoch: 3 [47104/60000 (78%)]	Loss: -0.974273
Train Epoch: 3 [47360/60000 (79%)]	Loss: -0.971570
Train Epoch: 3 [47616/60000 (79%)]	Loss: -0.970501
Train Epoch: 3 [47872/60000 (80%)]	Loss: -0.967825
Train Epoch: 3 [48128/60000 (80%)]	Loss: -0.973499
Train Epoch: 3 [48384/60000 (80%)]	Loss: -0.974183
Train Epoch: 3 [48640/60000 (81%)]	Loss: -0.964614
Train Epoch: 3 [48896/60000 (81%)]	Loss: -0.979683
Train Epoch: 3 [49152/60000 (82%)]	Loss: -0.967040
Train Epoch: 3 [49408/60000 (82%)]	Loss: -0.974914
Train Epoch: 3 [49664/60000 (83

...

Train Epoch: 101 [20480/60000 (34%)]	Loss: -1.647534
Train Epoch: 101 [20736/60000 (34%)]	Loss: -1.424652
Train Epoch: 101 [20992/60000 (35%)]	Loss: -1.579370
Train Epoch: 101 [21248/60000 (35%)]	Loss: -1.585732
Train Epoch: 101 [21504/60000 (36%)]	Loss: -1.630142
Train Epoch: 101 [21760/60000 (36%)]	Loss: -1.490448
Train Epoch: 101 [22016/60000 (37%)]	Loss: -1.630255
Train Epoch: 101 [22272/60000 (37%)]	Loss: -1.644259
Train Epoch: 101 [22528/60000 (37%)]	Loss: -1.461038
Train Epoch: 101 [22784/60000 (38%)]	Loss: -1.649256
Train Epoch: 101 [23040/60000 (38%)]	Loss: -1.624972
Train Epoch: 101 [23296/60000 (39%)]	Loss: -1.599450
Train Epoch: 101 [23552/60000 (39%)]	Loss: -1.669541
Train Epoch: 101 [23808/60000 (40%)]	Loss: -1.514402
Train Epoch: 101 [24064/60000 (40%)]	Loss: -1.650179
Train Epoch: 101 [24320/60000 (40%)]	Loss: -1.369860
Train Epoch: 101 [24576/60000 (41%)]	Loss: -1.598314
Train Epoch: 101 [24832/60000 (41%)]	Loss: -1.656197
Train Epoch: 101 [25088/60000 (42%)]	Loss: -1.

====> Test set loss: 2.9042
Train Epoch: 102 [0/60000 (0%)]	Loss: -1.662408
Train Epoch: 102 [256/60000 (0%)]	Loss: -1.642346
Train Epoch: 102 [512/60000 (1%)]	Loss: -1.641596
Train Epoch: 102 [768/60000 (1%)]	Loss: -1.663310
Train Epoch: 102 [1024/60000 (2%)]	Loss: -0.306491
Train Epoch: 102 [1280/60000 (2%)]	Loss: -1.607352
Train Epoch: 102 [1536/60000 (3%)]	Loss: -1.560975
Train Epoch: 102 [1792/60000 (3%)]	Loss: -1.523204
Train Epoch: 102 [2048/60000 (3%)]	Loss: -1.513249
Train Epoch: 102 [2304/60000 (4%)]	Loss: -1.522893
Train Epoch: 102 [2560/60000 (4%)]	Loss: -1.544507
Train Epoch: 102 [2816/60000 (5%)]	Loss: -1.590853
Train Epoch: 102 [3072/60000 (5%)]	Loss: -1.589195
Train Epoch: 102 [3328/60000 (6%)]	Loss: -1.535076
Train Epoch: 102 [3584/60000 (6%)]	Loss: -1.585638
Train Epoch: 102 [3840/60000 (6%)]	Loss: -1.452081
Train Epoch: 102 [4096/60000 (7%)]	Loss: -1.588794
Train Epoch: 102 [4352/60000 (7%)]	Loss: -1.587566
Train Epoch: 102 [4608/60000 (8%)]	Loss: -1.621429
Train Epo

Train Epoch: 102 [39936/60000 (66%)]	Loss: -1.566787
Train Epoch: 102 [40192/60000 (67%)]	Loss: -1.573129
Train Epoch: 102 [40448/60000 (67%)]	Loss: -1.600130
Train Epoch: 102 [40704/60000 (68%)]	Loss: -1.560816
Train Epoch: 102 [40960/60000 (68%)]	Loss: -1.434730
Train Epoch: 102 [41216/60000 (69%)]	Loss: -1.625731
Train Epoch: 102 [41472/60000 (69%)]	Loss: -1.618192
Train Epoch: 102 [41728/60000 (69%)]	Loss: -1.589017
Train Epoch: 102 [41984/60000 (70%)]	Loss: -1.607611
Train Epoch: 102 [42240/60000 (70%)]	Loss: -1.612610
Train Epoch: 102 [42496/60000 (71%)]	Loss: -1.586483
Train Epoch: 102 [42752/60000 (71%)]	Loss: -1.483295
Train Epoch: 102 [43008/60000 (71%)]	Loss: -1.607988
Train Epoch: 102 [43264/60000 (72%)]	Loss: -1.634552
Train Epoch: 102 [43520/60000 (72%)]	Loss: -1.624678
Train Epoch: 102 [43776/60000 (73%)]	Loss: -1.358419
Train Epoch: 102 [44032/60000 (73%)]	Loss: -1.634663
Train Epoch: 102 [44288/60000 (74%)]	Loss: -1.581809
Train Epoch: 102 [44544/60000 (74%)]	Loss: -1.

Train Epoch: 103 [19456/60000 (32%)]	Loss: -1.659277
Train Epoch: 103 [19712/60000 (33%)]	Loss: -1.597432
Train Epoch: 103 [19968/60000 (33%)]	Loss: -1.643137
Train Epoch: 103 [20224/60000 (34%)]	Loss: -1.566887
Train Epoch: 103 [20480/60000 (34%)]	Loss: -1.638420
Train Epoch: 103 [20736/60000 (34%)]	Loss: -1.635361
Train Epoch: 103 [20992/60000 (35%)]	Loss: -1.655505
Train Epoch: 103 [21248/60000 (35%)]	Loss: -1.641979
Train Epoch: 103 [21504/60000 (36%)]	Loss: -1.659711
Train Epoch: 103 [21760/60000 (36%)]	Loss: -1.651497
Train Epoch: 103 [22016/60000 (37%)]	Loss: -1.663548
Train Epoch: 103 [22272/60000 (37%)]	Loss: -1.636435
Train Epoch: 103 [22528/60000 (37%)]	Loss: -1.200102
Train Epoch: 103 [22784/60000 (38%)]	Loss: -1.653820
Train Epoch: 103 [23040/60000 (38%)]	Loss: -1.641896
Train Epoch: 103 [23296/60000 (39%)]	Loss: -1.579073
Train Epoch: 103 [23552/60000 (39%)]	Loss: -1.633989
Train Epoch: 103 [23808/60000 (40%)]	Loss: -1.637089
Train Epoch: 103 [24064/60000 (40%)]	Loss: -1.

KeyboardInterrupt: 

In [15]:
#save the model
save_path="./results_cnp_r_alea/model_param.pt"
torch.save(model.state_dict(), save_path)
